In [8]:
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import requests
from io import StringIO

#URL of the the raw CSV files from GitHub

url="https://raw.githubusercontent.com/TiagoAnalyst/Booking_system_dash-Python_Project/refs/heads/main/Data%20files/Recycling%20centre%20A%20-%20max.%20capacity.csv"
url2="https://raw.githubusercontent.com/TiagoAnalyst/Booking_system_dash-Python_Project/refs/heads/main/Data%20files/Recycling%20centre%20A%20-%20bookings.csv"


# Attempt to fetch the CSV files using requests
try:
    response = requests.get(url, timeout=10) # timeout
    response2 = requests.get(url2, timeout=10) #timeout increased

    url_data = response.content.decode('utf-8')
    url2_data =  response2.content.decode('utf-8')
    
    #Read the CSV data into a DataFrame
    BookingCapacity = pd.read_csv(StringIO(url_data))
    BookingDetails = pd.read_csv(StringIO(url2_data))

except requests.exceptions.RequestException as e:
    print(f"Error fetching the file: {e}")

#----------------------------------------
#Data preparation - Booking main dataset

BookingDetails = (
    BookingDetails
    .rename(
        columns={
            'Mattresses (from your home only)':'Mattresses',
            'Clothes and textiles (mixed)':'clothes and textiles',
            'Site':'Transport type',
        }
    )
    .assign(
        Booking_date = lambda x: pd.to_datetime(x['Booking date/time'], format='%d/%m/%Y  %H:%M').dt.strftime('%d/%m/%Y'),
        Booking_time = lambda y: pd.to_datetime(y['Booking date/time'], format='%d/%m/%Y  %H:%M').dt.strftime('%H:%M'),
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], dayfirst=True).dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], dayfirst=True).dt.month_name(),
        Booking_year= lambda x: pd.to_datetime(x['Booking_date'], dayfirst=True).dt.year,
        Booking_month_no = lambda i: pd.to_datetime(i['Booking_date'], format='%d/%m/%Y').dt.month,
        Financial_year = lambda k: k.apply(
            lambda t: f"{t['Booking_year']-1}-{t['Booking_year']}" if t['Booking_month_no']<=3 else f"{t['Booking_year']}-{t['Booking_year']+1}",
            axis=1
        )
    )
    .drop(columns=['Booking date/time'])
    .assign(
        **{'Booking created': pd.to_datetime(BookingDetails['Booking created'], format="%d/%m/%Y %H:%M", errors='coerce').dt.date}
    )
    .assign(
        **{
        'Transport type': lambda b: b['Transport type'].apply(lambda b: b[b.find('(')+1:b.find(')')]).str.lower()
        }
    )
)


#----------------------------------------
#Data preparation - Booking total capacity

BookingCapacity = (
    BookingCapacity
    .assign(
        **{
        'Vehicle type': lambda x: x['Vehicle type'].apply(lambda y: y[y.find('(')+1:y.find(')')]).str.lower()
        }
    )
    .rename(
        columns={
            'Date':'Booking_date'
        }
    )
)

#------------------------------------------
#Data preparation - Booking total capacity vs no. bookings made

BookingCapacity_total=BookingCapacity.groupby('Booking_date')['Number of spaces'].sum().reset_index(name="No. Bookings capacity")

temp_var=BookingDetails[BookingDetails['BookingStatus'].isin(['Completed','No show'])]
BookingDetails_attendance = temp_var.groupby('Booking_date')['BookingStatus'].count().reset_index(name="No. Bookings made")

BookingsDetails_summary = pd.merge(BookingCapacity_total,BookingDetails_attendance, left_on=None)

BookingsDetails_summary = (
    BookingsDetails_summary
        .assign(
            Booking_month = lambda a: pd.to_datetime(a['Booking_date'], dayfirst=True).dt.month_name()
    )
)

#--------------------------------------------
# Web app layout

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.PULSE])

app.layout = dbc.Container([
        dbc.Row([
            dbc.Col([
                    html.H1("Booking system dashboard - Financial year 2021-2022", style={'text-align':'center'}),
                    html.Br()
                ], width=12
                ) 
        ]),
        dbc.Row([
            dbc.Col([
                    html.Div('Please choose a month:'),
                    dcc.Dropdown(
                        id='slct_month',
                        options=[{'label': x, 'value': x} for x in BookingDetails['Booking_month'].unique()],
                        multi=False,
                        value= "April"
                    ),
                    html.Div(id='output_month',children=[])
                ],width=4
            )
        ]), 
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart3',
                            figure={}
                            )
            ], width=5),
            dbc.Col([
                dcc.Graph(id='chart1',
                            figure={}
                            )
            ], width=7)
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart2',
                            figure={} 
                            )
            ], width=6),
            dbc.Col([
                dcc.Graph(id='chart4',
                            figure={} 
                            )
            ] , width=6)
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart5',
                            figure={} 
                            )
            ], width=6)#,
            # dbc.Col([
            #     dcc.Graph(id='chart6',
            #                 figure={} 
            #                 )
            # ] , width=6)
        ])

])

#container_month, Bookings_capacity, closure_reason, gauge_chart, average_bookings
    
    #https://youtu.be/1nEL0S8i2Wk?si=hCCh51-MG2zYgHVi

#--------------------------------------------------------------------
#Connect the Plotly graphs with Dash Components -- callback
@app.callback(
    [
        Output(component_id='output_month',component_property='children'),
        Output(component_id='chart1',component_property='figure'),
        Output(component_id='chart2',component_property='figure'),
        Output(component_id='chart3',component_property='figure'),
        Output(component_id='chart4',component_property='figure'),
        Output(component_id='chart5',component_property='figure')
        #Output(component_id='chart6',component_property='figure')
    ],
    [
        Input(component_id='slct_month', component_property='value')
    ]
)

def update_graphs(slct_month): 
    print(type(slct_month))

    #container

    container_month = f"The month chosen by user was: {slct_month}"

    #filter data based on selected month
    BookingsDetails_summary_filtered = BookingsDetails_summary[BookingsDetails_summary['Booking_month'] == slct_month]
    BookingsDetails_summary_filtered.loc[:,'Booking_date'] = pd.to_datetime(BookingsDetails_summary_filtered['Booking_date'],format='%d/%m/%Y', dayfirst=True)

   #Chart 1 - no.bookings vs total capacity

    fig1 = make_subplots(specs=[[{"secondary_y": True}]])
    fig1.add_trace(
        go.Bar(
            x=BookingsDetails_summary_filtered['Booking_date'],
            y=BookingsDetails_summary_filtered['No. Bookings made'],
            name='No. of bookings'
        ),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(
            x=BookingsDetails_summary_filtered['Booking_date'],
            y=BookingsDetails_summary_filtered['No. Bookings capacity'],
            name='Daily capacity',
            ),
        secondary_y=True
    )

        # Chart configuration
    fig1.update_xaxes(title_text="Date")\
        .update_yaxes(title_text="<b>Bookings capacity</b>", secondary_y=False)\
        .update_yaxes(title_text="<b>No. of bookings</b>", secondary_y=True)\
        .update_yaxes(range = [0,500])\
        .update_xaxes(
            tickformat='%d',
            tickmode='array',
            tickvals=BookingsDetails_summary_filtered['Booking_date'])\
        .update_layout(
            title_text = "Number of bookings made vs bookings capacity",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

     # Chart2 - Booking Status KPIs
    
    bookings_classification = BookingDetails[BookingDetails['Booking_month'] == slct_month]['BookingStatus'].value_counts().reset_index()
    bookings_classification.columns=['BookingStatus','Count']
        
    fig2 =px.pie(bookings_classification, 
                           names = 'BookingStatus',
                           values='Count',
                           title='Booking Status distribution'
            )
    
    fig2.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1))
    
    # Chart3 - Gauge chart for booking capacity usage

    Gauge_chart = (BookingsDetails_summary_filtered['No. Bookings made'].sum()/BookingsDetails_summary_filtered['No. Bookings capacity'].sum())*100

    fig3 = go.Figure(
            go.Indicator(
                mode= "gauge+number",
                value=Gauge_chart,
                number = {'suffix':"%"},
                domain={'x':[0,1],'y':[0,1]},
                title ={'text':"Bookings capacity usage"},
                gauge ={'axis': {'range': [0,100]},
                        'bar':{'color':'grey'},
                        'steps':[
                            {'range':[0,80],'color':"lightgreen"},
                            {'range':[80,90],'color':"yellow"},
                            {'range':[90,100],'color':"red"}
                            ]
                }
            )
        )

    # Chart4 - busiest weekdays 

    Ordered_weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    BookingDetails_filtered = BookingDetails[BookingDetails['Booking_month'] == slct_month]

    Bookings_dayofweek_list=[]

    for i in Ordered_weekday:
        var_temp = BookingDetails_filtered.loc[
            (BookingDetails_filtered['Booking_dayofweek']==i) & 
            (BookingDetails_filtered['BookingStatus'].isin(['No show','Completed']))
        ].copy()
        if not var_temp.empty:
            var_temp2 = var_temp['BookingStatus'].count()/var_temp['Booking_date'].nunique()
        else:
            var_temp2=0

        Bookings_dayofweek_list.append({'Booking_dayofweek':i,'Avg_bookings': var_temp2})

    Bookings_dayofweek_table = pd.DataFrame(Bookings_dayofweek_list)

        #determine the day with the highest average
    max_avg_booking_day = Bookings_dayofweek_table.loc[Bookings_dayofweek_table['Avg_bookings'].idxmax(),'Booking_dayofweek']

    #colours

    colors=['blue']*len(Bookings_dayofweek_table)
    max_day_index = Bookings_dayofweek_table.index[Bookings_dayofweek_table['Booking_dayofweek']== max_avg_booking_day].tolist()[0]
    colors[max_day_index] = 'crimson'

    name_short = lambda x: x[:3]
    Weekday_short = [name_short(day) for day in Ordered_weekday]


    fig4 = go.Figure(
        go.Bar(
                x=Weekday_short,
                y=Bookings_dayofweek_table['Avg_bookings'],
                marker_color = colors
        )
    )

    fig4.update_layout(title_text='Average no. of bookings per day of week')

    # Chart 5 - most frequent visitors

    most_visitors = BookingDetails[BookingDetails['Booking_month'] == slct_month]['Registration plate'].value_counts().reset_index()
    most_visitors.columns=['Registration plate','No. of visits']

    fig5 = go.Figure(
        data=[go.Table(
            header=dict(values=list(most_visitors.columns),
                        fill_color='paleturquoise',
                        align='left'),
            cells=dict(values=[most_visitors['Registration plate'],most_visitors['No. of visits']],
                       fill_color='lavender',
                       align='left')
        )]
    )

    return container_month, fig1, fig2, fig3, fig4, fig5
#--------------------------------------------------------------------

if __name__=='__main__':
    app.run_server(debug=True)

<class 'str'>


C:\Users\Tiago\AppData\Local\Temp\ipykernel_13652\4051076819.py:262: RuntimeWarning:

invalid value encountered in scalar divide



<class 'str'>


C:\Users\Tiago\AppData\Local\Temp\ipykernel_13652\4051076819.py:262: RuntimeWarning:

invalid value encountered in scalar divide



### Testing ground - Datasets loaded and code up to the end of data preparation - DONT USE SECTION BELOW

In [21]:
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import requests
from io import StringIO

#URL of the the raw CSV files from GitHub

url="https://raw.githubusercontent.com/TiagoAnalyst/Booking_system_dash-Python_Project/refs/heads/main/Data%20files/Recycling%20centre%20A%20-%20max.%20capacity.csv"
url2="https://raw.githubusercontent.com/TiagoAnalyst/Booking_system_dash-Python_Project/refs/heads/main/Data%20files/Recycling%20centre%20A%20-%20bookings.csv"


# Attempt to fetch the CSV files using requests
try:
    response = requests.get(url, timeout=10) # timeout
    response2 = requests.get(url2, timeout=10) #timeout increased

    url_data = response.content.decode('utf-8')
    url2_data =  response2.content.decode('utf-8')
    
    #Read the CSV data into a DataFrame
    BookingCapacity = pd.read_csv(StringIO(url_data))
    BookingDetails = pd.read_csv(StringIO(url2_data))

except requests.exceptions.RequestException as e:
    print(f"Error fetching the file: {e}")

#----------------------------------------
#Data preparation - Booking main dataset

BookingDetails = (
    BookingDetails
    .rename(
        columns={
            'Mattresses (from your home only)':'Mattresses',
            'Clothes and textiles (mixed)':'clothes and textiles',
            'Site':'Transport type',
        }
    )
    .assign(
        Booking_date = lambda x: pd.to_datetime(x['Booking date/time'], format='%d/%m/%Y  %H:%M').dt.strftime('%d/%m/%Y'),
        Booking_time = lambda y: pd.to_datetime(y['Booking date/time'], format='%d/%m/%Y  %H:%M').dt.strftime('%H:%M'),
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], dayfirst=True).dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], dayfirst=True).dt.month_name(),
        Booking_year= lambda x: pd.to_datetime(x['Booking_date'], dayfirst=True).dt.year,
        Booking_month_no = lambda i: pd.to_datetime(i['Booking_date'], format='%d/%m/%Y').dt.month,
        Financial_year = lambda k: k.apply(
            lambda t: f"{t['Booking_year']-1}-{t['Booking_year']}" if t['Booking_month_no']<=3 else f"{t['Booking_year']}-{t['Booking_year']+1}",
            axis=1
        )
    )
    .drop(columns=['Booking date/time'])
    .assign(
        **{'Booking created': pd.to_datetime(BookingDetails['Booking created'], format="%d/%m/%Y %H:%M", errors='coerce').dt.date}
    )
    .assign(
        **{
        'Transport type': lambda b: b['Transport type'].apply(lambda b: b[b.find('(')+1:b.find(')')]).str.lower()
        }
    )
)


#----------------------------------------
#Data preparation - Booking total capacity

BookingCapacity = (
    BookingCapacity
    .assign(
        **{
        'Vehicle type': lambda x: x['Vehicle type'].apply(lambda y: y[y.find('(')+1:y.find(')')]).str.lower()
        }
    )
    .rename(
        columns={
            'Date':'Booking_date'
        }
    )
)

#------------------------------------------
#Data preparation - Booking total capacity vs no. bookings made



In [23]:
vartemp = BookingDetails_attendance.groupby('Booking_date')['BookingStatus'].count().reset_index(name="No. Bookings made")

KeyError: 'Column not found: BookingStatus'

In [ ]:
BookingCapacity_total=BookingCapacity.groupby('Booking_date')['Number of spaces'].sum().reset_index(name="No. Bookings capacity")

    # merge of both tables - final table

vartemp = BookingDetails_attendance.groupby('Booking_date')['BookingStatus'].count().reset_index(name="No. Bookings made")

BookingsDetails_Capacity = pd.merge(
    BookingCapacity_total,
    vartemp, 
    left_on=None)

BookingsDetails_Capacity = (
    BookingsDetails_Capacity
        .assign(
            Booking_month = lambda a: pd.to_datetime(a['Booking_date'], dayfirst=True).dt.month_name(),
            Booking_year= lambda b: pd.to_datetime(b['Booking_date'], dayfirst=True).dt.year,
            Booking_month_no = lambda i: pd.to_datetime(i['Booking_date'], format='%d/%m/%Y').dt.month,
            Financial_year = lambda y:y.apply(
                lambda t: f"{t['Booking_year']-1}-{t['Booking_year']}" if t['Booking_month_no']<4 else f"{t['Booking_year']}-{t['Booking_year']+1}",
            axis=1
            )
        )
)

# To use this space, first run the previous script and use this one afterwards

In [4]:
slct_month="April"

In [18]:
BookingCapacity_total

,Booking_date,No. Bookings capacity
0,01/02/2022,380
1,01/03/2022,380
2,01/04/2021,352
3,01/05/2021,352
4,01/06/2021,352
...,...,...
358,31/05/2021,380
359,31/07/2021,352
360,31/08/2021,352
361,31/10/2021,380


In [19]:
BookingDetails_attendance

,Booking_date,No. Bookings made
0,01/02/2024,162
1,01/03/2024,95
2,01/04/2023,155
3,01/05/2023,308
4,01/06/2023,321
...,...,...
307,31/03/2024,171
308,31/07/2023,158
309,31/08/2023,305
310,31/10/2023,164


In [10]:
Gauge_chart = (BookingsDetails_summary['No. Bookings made'].sum()/BookingsDetails_summary['No. Bookings capacity'].sum())*100

fig3 = go.Figure(
        go.Indicator(
            mode= "gauge+number",
            value=Gauge_chart,
            number = {'suffix':"%"},
            domain={'x':[0,1],'y':[0,1]},
            title ={'text':"Bookings capacity usage"},
            gauge ={'axis': {'range': [0,100]},
                    'bar':{'color':'grey'},
                    'steps':[
                        {'range':[0,80],'color':"lightgreen"},
                        {'range':[80,90],'color':"yellow"},
                        {'range':[90,100],'color':"red"}
                        ]
            }
        )
    )

C:\Users\Tiago\AppData\Local\Temp\ipykernel_13652\64151901.py:1: RuntimeWarning:

invalid value encountered in scalar divide

